In [1]:
import pandas as pd
import numpy as np
import json
import spacy
import pathlib

In [47]:
from sentence_transformers import SentenceTransformer, util
transformer=SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
# model = SentenceTransformer('paraphrase-MiniLM-L12-v2')
def get_tensor(sentence):
    transformer=SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    return transformer.encode(sentence,convert_to_tensor=True)
def encode_questions(df):
    df['q_tensor']=df.q.transform(get_tensor)
    return df
def encode_answers(df):
    df['a_tensor']=df.a.transform(get_tensor)
    return df

In [48]:
cwd_path=pathlib.Path().cwd()
data_path=cwd_path.joinpath('wirtschaftsinformatik-main/business-intelligence.json')
with open(data_path,encoding='utf-8') as f:
  data = json.load(f)

In [ ]:
dftest=pd.DataFrame()

In [2]:
cwd_path=pathlib.Path().cwd()
data_path=cwd_path.joinpath('wirtschaftsinformatik-main')

In [3]:
deck_files=list(data_path.glob(r'*.json'))

In [4]:
deck_files

[WindowsPath('C:/Users/lekle/Documents/Projects/Fancy_Flashcards_Bot_project/wirtschaftsinformatik-main/business-intelligence.json'),
 WindowsPath('C:/Users/lekle/Documents/Projects/Fancy_Flashcards_Bot_project/wirtschaftsinformatik-main/einfuehrung-in-die-wirtschaftsinformatik.json'),
 WindowsPath('C:/Users/lekle/Documents/Projects/Fancy_Flashcards_Bot_project/wirtschaftsinformatik-main/Finanzbuchhaltung.json'),
 WindowsPath('C:/Users/lekle/Documents/Projects/Fancy_Flashcards_Bot_project/wirtschaftsinformatik-main/finanzierung-und-investition.json'),
 WindowsPath('C:/Users/lekle/Documents/Projects/Fancy_Flashcards_Bot_project/wirtschaftsinformatik-main/IT-Recht.json'),
 WindowsPath('C:/Users/lekle/Documents/Projects/Fancy_Flashcards_Bot_project/wirtschaftsinformatik-main/IT-Security.json'),
 WindowsPath('C:/Users/lekle/Documents/Projects/Fancy_Flashcards_Bot_project/wirtschaftsinformatik-main/IT-Sicherheit-II.json'),
 WindowsPath('C:/Users/lekle/Documents/Projects/Fancy_Flashcards_Bot

In [59]:
transformer=SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
frames=[]
for file in deck_files:
#     print(file)
    topic=file.stem
    with open(file,encoding='utf-8') as f:
        data = json.load(f)
    decks=data['decks']
    for deck in decks:
#         print(deck)
        cards=decks[deck]['cards']
        df=pd.DataFrame(cards)
        df=df.transpose()
        df=df.assign(topic=[topic for x in range(len(df.index))],deck=[deck for x in range(len(df.index))])
#         df['topic']=pd.Series([topic for x in range(len(df.index))], index=df.index)
#         df['deck']=pd.Series([deck for x in range(len(df.index))], index=df.index)
        df['q_tensor']=df.q.transform(get_tensor)
        df['a_tensor']=df.a.transform(get_tensor)
        frames.append(df)
        print('deck finished:',deck)
    print('topic finished:',topic)
dataframe=pd.concat(frames,axis=0)
# dataframe=encode_questions(dataframe)
# dataframe=encode_answers(dataframe)

deck finished: d1
topic finished: business-intelligence
deck finished: d1
topic finished: einfuehrung-in-die-wirtschaftsinformatik
deck finished: d0
topic finished: Finanzbuchhaltung
deck finished: d1
topic finished: finanzierung-und-investition
deck finished: d0
deck finished: d1
deck finished: d2
deck finished: d3
deck finished: d4
deck finished: d5
deck finished: d6
deck finished: d7
deck finished: d8
topic finished: IT-Recht
deck finished: d0
topic finished: IT-Security
deck finished: IT-Sicherheit II
topic finished: IT-Sicherheit-II
deck finished: itil
topic finished: ITIL
deck finished: d0
deck finished: d1
topic finished: kommunikationssysteme
deck finished: d1
topic finished: open-source-contributor-skills
deck finished: d1
topic finished: personalmanagement
deck finished: d0
topic finished: unternehmensfuehrung


In [69]:
dataframe.to_csv('data.csv',sep=';',index=False)

In [65]:
dataframe['q_tensor']=dataframe.q_tensor.transform(lambda x: x.tolist())
dataframe['a_tensor']=dataframe.q_tensor.transform(lambda x: x.tolist())

In [68]:
dataframe.head()

,q,a,topic,deck,q_tensor,a_tensor
0,Was führt zu einer zunehmenden Datenflut/ Big ...,"Zunehmende Digitalisierung (Scannerkassen, Tra...",business-intelligence,d1,"[-0.15225397050380707, -0.1358981877565384, -0...","[-0.15225397050380707, -0.1358981877565384, -0..."
1,Wodurch wird ein Mehrwert generiert?,"durch Analyse der Datenflut, d.h. Daten -> Inf...",business-intelligence,d1,"[-0.014535852707922459, 0.13996674120426178, -...","[-0.014535852707922459, 0.13996674120426178, -..."
2,Was versteht man unter Business Intelligence (...,Sammelbegriff für den IT-gestützten Zugriff au...,business-intelligence,d1,"[-0.21210268139839172, 0.15913116931915283, -0...","[-0.21210268139839172, 0.15913116931915283, -0..."
3,Was ist das Ziel von BI (Lackes/ Siepermann)?,Generierung von neuem Wissen aus dem bereits v...,business-intelligence,d1,"[-0.13337299227714539, 0.4409675598144531, -0....","[-0.13337299227714539, 0.4409675598144531, -0...."
4,Was ist bei dem neuen Wissen zu beachten (Lack...,Wissen sollte relevant und handlungsorientiert...,business-intelligence,d1,"[-0.05927468463778496, -0.029863983392715454, ...","[-0.05927468463778496, -0.029863983392715454, ..."


In [70]:
from sqlalchemy import create_engine
db_file=cwd_path.joinpath('cards.db')

engine = create_engine('sqlite:///'+db_file.as_posix(), echo=False)
dataframe = dataframe.applymap(str)
dataframe.to_sql('cards', con=engine,if_exists='replace')

In [39]:
decks=data['decks']
# print('Number of decks: ',len(decks))
cards=decks['d1']['cards']
df=pd.DataFrame(cards)
df=df.transpose()

KeyError: 'd1'

In [4]:
df.head()

,q,a
0,Was führt zu einer zunehmenden Datenflut/ Big ...,"Zunehmende Digitalisierung (Scannerkassen, Tra..."
1,Wodurch wird ein Mehrwert generiert?,"durch Analyse der Datenflut, d.h. Daten -> Inf..."
2,Was versteht man unter Business Intelligence (...,Sammelbegriff für den IT-gestützten Zugriff au...
3,Was ist das Ziel von BI (Lackes/ Siepermann)?,Generierung von neuem Wissen aus dem bereits v...
4,Was ist bei dem neuen Wissen zu beachten (Lack...,Wissen sollte relevant und handlungsorientiert...


# spacy de_core_news_lg

In [5]:
df.a.iloc[0]

'Zunehmende Digitalisierung (Scannerkassen, Transaktionen im Internet, Social Media, Activity Tracker, Wearables, Smart-Home-Systeme)'

In [7]:
nlp=spacy.load('de_core_news_lg')
STOP_WORDS = spacy.lang.de.stop_words.STOP_WORDS

In [12]:
df.a.iloc[0]
a='Grund dafür sind Social Media,Activity Tracker und Transaktionen im Internet. Die zunehmende Digitalisierung führt zur zunehmenden Datenflut.'
doc3=nlp(a)

In [10]:
doc1=nlp(df.a.iloc[0])
doc2=nlp(df.a.iloc[1])

In [13]:
doc1.similarity(doc3)

0.5983387920746743

In [14]:
index=0
answer=df.a.iloc[index]
similarities=[nlp(answer).similarity(nlp(df.a.iloc[x])) for x in range(len(df))]

ipykernel_launcher:3: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


In [15]:
similarities[8]

0.358827437260183

In [16]:
for i,similarity in enumerate(similarities):

    if similarity >= 0.75:
        print(i,'Gut gemacht! \n',answer)
    else:
        # print(i,'Schade!', answer)
        continue

0 Gut gemacht! 
 Zunehmende Digitalisierung (Scannerkassen, Transaktionen im Internet, Social Media, Activity Tracker, Wearables, Smart-Home-Systeme)
11 Gut gemacht! 
 Zunehmende Digitalisierung (Scannerkassen, Transaktionen im Internet, Social Media, Activity Tracker, Wearables, Smart-Home-Systeme)
158 Gut gemacht! 
 Zunehmende Digitalisierung (Scannerkassen, Transaktionen im Internet, Social Media, Activity Tracker, Wearables, Smart-Home-Systeme)
160 Gut gemacht! 
 Zunehmende Digitalisierung (Scannerkassen, Transaktionen im Internet, Social Media, Activity Tracker, Wearables, Smart-Home-Systeme)


# tfidf via scikit-learn

In [85]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from HanTa import HanoverTagger as ht
import string
# nltk.download()

In [61]:
a=nltk.tokenize.word_tokenize(df.a.iloc[0],language='german')
b=ht.HanoverTagger('morphmodel_ger.pgz').tag_sent(a)

In [66]:
word,lemma,pos=b[6]

In [67]:
lemma

'im'

In [123]:
def preprocess(text):
    # tagger=HanTa.HanoverTagger('morphmodel_ger.pgz')
    a=ht.HanoverTagger('morphmodel_ger.pgz').tag_sent([text])
    # print(a)
    # print(len(a))
    return a[0]
def tokenizer(text):
    tokens=nltk.tokenize.word_tokenize(text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))),language='german')
    lemmas=[]
    # print(tokens)
    for token in tokens:
        word,lemma,pos=preprocess(token)
        lemmas.append(lemma)
    return lemmas

In [106]:
tagger=ht.HanoverTagger('morphmodel_ger.pgz')
tagger.tag_sent(['lief','Türen'])

[('lief', 'laufen', 'VVFIN'), ('Türen', 'Tür', 'NN')]

In [48]:
xtrain,xtest,ytrain,ytest=train_test_split(df.q,df.a,test_size=0.33, random_state=1)

In [126]:
stopwords=nltk.corpus.stopwords.words('german')
stemmer=nltk.stem.snowball.SnowballStemmer('german',ignore_stopwords=True)

tfidf=TfidfVectorizer(tokenizer=tokenizer)
transformer=tfidf.fit(xtrain)
xtraint=transformer.transform(xtest)
vocab=transformer.vocabulary_

C:\Users\lekle\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


KeyboardInterrupt: 

In [128]:
vocab=transformer.vocabulary_b

{'wie': 327,
 'gestalten': 256,
 'sich': 295,
 'die': 227,
 'Zeitaufteilung': 187,
 'des': 226,
 'Datenanalyseprozesses': 35,
 'zu': 335,
 'welchen': 321,
 'statistisch': 301,
 'Fehler': 68,
 'kommen': 269,
 'es': 244,
 'bei': 210,
 'der': 225,
 'Datenerhebung': 37,
 'häufig': 262,
 'was': 318,
 'sein': 294,
 'unter': 305,
 'einer': 234,
 'Clusteranalyse': 28,
 'Verstehen': 178,
 'Nachteil': 115,
 'Vollerhebung': 179,
 'ein': 231,
 'wichtig': 326,
 'Distanzmaß': 47,
 'für': 249,
 'Metrisch': 108,
 'Skalierte': 145,
 'variabel': 308,
 'im': 264,
 'Kontext': 93,
 'auf': 203,
 'welche': 320,
 'Weise': 185,
 'können': 272,
 'Erhebung': 59,
 'Von': 180,
 'Primärdaten': 122,
 'Erfolg': 58,
 'Aussagekraft': 11,
 'T': 163,
 'Test': 167,
 'Vorgehensweise': 182,
 'Conjoint': 31,
 'Analyse': 5,
 'beschreiben': 215,
 'sie': 296,
 'Erstellung': 62,
 'Standarddatenformats': 149,
 'Zug': 192,
 'Datenaufbereitung': 36,
 'nennen': 281,
 'ausgewählt': 206,
 'Verfahren': 176,
 'rein': 291,
 'Zeitreihenan

# sentence_transformer

In [5]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-MiniLM-L12-v2')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))


The cat sits outside 		 The dog plays in the garden 		 Score: 0.1927
A man is playing guitar 		 A woman watches TV 		 Score: 0.0188
The new movie is awesome 		 The new movie is so great 		 Score: 0.9292


In [6]:
transformer=SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [86]:
a='Die zunehmende Digitalisierung führt zur zunehmenden Datenflut. Social Media'
e1=transformer.encode(df.a.iloc[0],convert_to_tensor=True)
e2=transformer.encode(a,convert_to_tensor=True)

In [87]:
cosine_scores = util.pytorch_cos_sim(e1, e2)

In [88]:
cosine_scores.tolist()[0]

[0.7355790734291077]

In [26]:
e_a=transformer.encode(df.a,convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(e_a, e_a)

In [32]:
def get_tensor(sentence):
    return transformer.encode(sentence,convert_to_tensor=True)

In [33]:
df['tensor']=df.a.transform(get_tensor)

In [38]:
for x,i in enumerate(df.tensor.head()):
    print(x,i)

0 tensor([-3.6298e-02, -8.8171e-02,  4.5257e-03, -5.9105e-02,  2.6965e-01,
         2.9514e-02, -2.0731e-01, -2.0581e-01, -2.3975e-02,  3.3644e-01,
         1.3636e-01,  3.3719e-01,  3.3671e-01, -2.0095e-02,  2.1650e-01,
        -8.1841e-02,  4.7340e-02, -3.3699e-02,  1.7373e-01, -8.0975e-02,
        -9.3431e-02, -3.7411e-01,  5.4197e-02, -1.8535e-01,  4.2887e-02,
        -7.4687e-02, -1.2232e-01, -1.0156e-01, -2.0580e-01, -3.0818e-01,
         2.2100e-01, -2.3205e-02,  2.8332e-01,  1.2833e-01, -1.4248e-01,
        -1.0321e-01,  3.6045e-03, -1.4820e-01, -1.0847e-01, -2.0706e-01,
        -1.9448e-01, -2.0881e-01, -8.6197e-02,  5.9866e-03,  6.0061e-02,
         4.1313e-02, -4.7154e-02,  1.3740e-01, -3.2033e-01, -2.2846e-02,
         1.1127e-02, -1.5561e-01, -9.4863e-03,  4.2842e-01,  2.0535e-02,
         1.6417e-01,  7.0818e-02,  1.7185e-01, -9.5212e-02,  8.2671e-03,
         2.0868e-01, -1.0497e-03, -1.6931e-01,  3.4725e-01,  6.1675e-02,
         3.5668e-01, -2.2412e-02,  3.6954e-02, -1

In [56]:
for t in df.tensor:
    print(type(t))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'tor

In [44]:
cosine_scores = util.pytorch_cos_sim(df.tensor.iloc[0], df.tensor.iloc[1])

In [61]:
def get_similarities(tensor):
    cosine_scores=[]
    for t in df.tensor:
        cosine_score = util.pytorch_cos_sim(tensor, t)
        cosine_scores.append(cosine_score.tolist()[0])
    return cosine_scores

In [62]:
df['similarities']=df.tensor.transform(get_similarities)

In [92]:
np.argmax(df.similarities.iloc[4])

4

In [75]:
ind=df.index.tolist()
# ind=[str(x) for x in ind]

In [76]:
sims=pd.DataFrame(df.similarities.tolist(),columns=ind)

In [79]:
sims.head()

,0,1,2,3,4,5,6,7,8,9,...,217,218,219,220,221,222,223,224,225,226
0,[1.0000001192092896],[0.3842788636684418],[0.4646209478378296],[0.3520452380180359],[0.2724843919277191],[0.4735337495803833],[0.23634560406208038],[0.5132248401641846],[0.3701647222042084],[0.4184598922729492],...,[0.12146133929491043],[0.14966003596782684],[0.09578536450862885],[0.13936761021614075],[0.2662913203239441],[0.06528446823358536],[0.00818929448723793],[0.11995619535446167],[0.12486793845891953],[0.2702973484992981]
1,[0.3842788636684418],[0.9999997019767761],[0.697367250919342],[0.5421848297119141],[0.487244576215744],[0.4778689742088318],[0.48688337206840515],[0.7250967025756836],[0.714789092540741],[0.7993707656860352],...,[0.2835153639316559],[0.2931622564792633],[0.3415345251560211],[0.32081085443496704],[0.39762601256370544],[0.22260142862796783],[0.19439035654067993],[0.3026965260505676],[0.285799503326416],[0.5848177671432495]
2,[0.4646209478378296],[0.697367250919342],[0.9999999403953552],[0.4677650034427643],[0.45360085368156433],[0.5945021510124207],[0.3495512902736664],[0.6469554901123047],[0.5802361965179443],[0.6547772884368896],...,[0.13600660860538483],[0.129466250538826],[0.2250801920890808],[0.20253324508666992],[0.21864677965641022],[0.06637073308229446],[0.03670518100261688],[0.16445568203926086],[0.10163886100053787],[0.43913355469703674]
3,[0.3520452380180359],[0.5421848297119141],[0.4677650034427643],[0.9999996423721313],[0.6722910404205322],[0.5442690849304199],[0.20280784368515015],[0.5330365300178528],[0.43110623955726624],[0.6423788666725159],...,[0.07511422783136368],[0.1541917771100998],[0.18107576668262482],[0.19329868257045746],[0.30590033531188965],[0.04909500852227211],[0.03288514167070389],[0.10561758279800415],[0.03432900458574295],[0.3298330008983612]
4,[0.2724843919277191],[0.487244576215744],[0.45360085368156433],[0.6722910404205322],[1.0000001192092896],[0.6854268312454224],[0.14749059081077576],[0.5178467631340027],[0.34635308384895325],[0.4633464515209198],...,[0.041354041546583176],[0.1315627247095108],[0.1388622373342514],[0.11807291954755783],[0.3429238796234131],[-0.027955174446105957],[-0.06809111684560776],[0.010115067474544048],[0.05892864987254143],[0.4474967420101166]
